In [1]:
import exploding_kittens as expl
import random

# GAME MECHANICS

##### 1. START OF TURN:
- PLAYER THINKING -> PLAYER ACTION
- DISCARD PILE UPDATED
- DRAW (END TURN)

In [51]:
class Game:
    """
    Class for the main functionality of the game. Upon initialisation,
    the deck is created and each player is dealt a hand of 5 cards
    plus one Defuse card. Then the remaining Defuse cards and the
    Exploding Kittens are shuffled into the deck.
    """

    def __init__(self, players=("Player 1", "Player 2")):
        self.deck = expl.Deck(num_of_players=len(players))
        self.discard_pile = []
        self.dead_players = []
        self.players = []
        for pl in players:
            self.players.append(expl.Player(pl))
            self.players[-1].deal_hand(
                [self.deck.cards.pop(0) for _ in range(5)] + [expl.Defuse()]
            )
            self.players[-1].hand.sort(key=lambda x: x.id)
        self.deck.add_defuse_and_exploding_to_deck()

    def update_discard_pile(self, played_card):
        if played_card is not None:
            self.discard_pile += [played_card]

    def execute_all_turns_for_a_player(self):
        """
        For the current player:

            while the player is still required to draw:
                while the player wants to play a card:
                    - Choose what card to play
                    - If a card is chosen
                        - Update the discard pile
                        - Perform the card's action
                    - Otherwise, set the stopping criteria to True
                - The player draws a card to end their turn
                - Check if the player exploded and resolve it
        """
        while self.cards_to_draw != 0:
            end_turn = False

            while not end_turn:
                played_card = self.current_player.choose_card_to_play()
                if played_card is not None:
                    print(f"Player {self.current_player} plays {played_card}")
                    input()
                    self.update_discard_pile(played_card)
                    played_card.action(self)  # Pass the game class into the Cards class
                else:
                    end_turn = True
            
            # TODO: Need a check here for the case of playing a Skip or an Attack
            # e.g. When a player has 1 more card to draw and plays a Skip, 
            # should we set self.cards_to_draw=0?
            # Similarly if the same player plays an Attack, should we set
            # self.cards_to_draw = 2 OR self.next_player_cards_to_draw = 0
            
            drawn_card = self.current_player.draw_to_end_turn(self)
            print(f"Player {self.current_player} draws {drawn_card}")
            input()
            if drawn_card.id == 0:
                exploded = self.current_player.explode()
                if exploded:
                    self.dead_players.append(self.current_player)
                    self.players.pop(self.current_player.id)
                    self.update_discard_pile(drawn_card)
                    print(f"Bye bye player {self.current_player}")
                    input()

            self.cards_to_draw -= 1


    def play_game(self):
        end_of_game = False
        self.turn_count = 0
        self.cards_to_draw = 1
        while not end_of_game:
            player_id = self.turn_count % len(self.players)
            self.current_player = self.players[player_id]
            self.current_player.id = player_id

            self.execute_all_turns_for_a_player()
            
            # TODO: Remove this line
            self.cards_to_draw = 1
            
            if len(self.players) == 1:
                end_of_game = True

            self.turn_count += 1


In [52]:
game = Game(players=["Mixas", "Vasos", "Ponjos"])

In [53]:
game.play_game()

Player Mixas draws Beard Cat
Player Vasos draws Attack
Player Ponjos plays See The Future
Player Ponjos draws Catermelon
Player Mixas draws Beard Cat
Player Vasos draws See The Future
Player Ponjos draws Defuse
Player Mixas plays Defuse
Player Mixas draws Exploding Kitten
Bye bye player Mixas
Player Ponjos draws Nope
Player Vasos draws Favor
Player Ponjos draws Favor
Player Vasos draws Hairy Potato Cat
Player Ponjos draws Attack
Player Vasos draws Tacocat
Player Ponjos draws Beard Cat
Player Vasos plays Defuse
Player Vasos plays Nope
Player Vasos draws Nope
Player Ponjos draws See The Future
Player Vasos draws Shuffle
Player Ponjos draws Rainbow Cat
Player Vasos draws Attack
Player Ponjos draws Nope
Player Vasos draws Shuffle
Player Ponjos draws Rainbow Cat
Player Vasos plays Attack
Player Vasos draws Skip
Player Ponjos plays See The Future
Player Ponjos draws Exploding Kitten
Bye bye player Ponjos


In [40]:
game.players

[Vasos]

In [41]:
game.dead_players

[Ponjos, Mixas]

In [42]:
game.discard_pile

[Attack, Tacocat, Defuse, Catermelon, Exploding Kitten, Exploding Kitten]

In [43]:
game.dead_players[1].hand

[Catermelon, Favor, Nope, Shuffle, See The Future]

In [44]:
game.players[0].hand

[Defuse,
 Hairy Potato Cat,
 Beard Cat,
 Skip,
 See The Future,
 See The Future,
 Defuse,
 Rainbow Cat]